In [26]:
# import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
# %matplotlib inline

In [27]:
import datetime
import pickle as cPickle
import csv
import numpy as np
import random
import sys
from file_util import create_folder
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [28]:
# name_dataset        = 'news-19_paragraph_swap-random-1m'
# name_dataset        = 'headline_swap_news_v2'
# name_dataset        = 'headline_swap_news_v2.5'
# name_dataset        = 'headline_swap_news_v2.5_mf8'
name_dataset        = 'headline'
path_raw_data       = '../data/raw/' + name_dataset + '/'
path_processed_data = '../data/' + name_dataset + '/whole/'

In [29]:
create_folder(path_raw_data)
create_folder(path_processed_data + '/train')
create_folder(path_processed_data + '/dev')
create_folder(path_processed_data + '/test')
create_folder(path_processed_data + '/debug')
create_folder(path_processed_data + '/real_world')

In [5]:
print(path_raw_data)

../data/raw/headline/


In [6]:
print (path_processed_data)

../data/headline/whole/


# get term-/document-frequency

In [30]:
csv_reader = csv.reader(open(path_raw_data + 'train.csv', encoding='utf-8'))

tkn2tf = {}
len_heads = [] #1
len_paras = [] #2
cnt_paras = [] #3
len_bodys = [] #4

# csv data: 0:id, 1:head, 2:body, 3:label

print (datetime.datetime.now().isoformat())

for n, row in enumerate(csv_reader):
    if (n+1) % 100000 == 0: print (n+1),
    
    head = row[1].lower().strip()
    
    for tkn in head.split():
        if tkn in tkn2tf: tkn2tf[tkn] += 1
        else: tkn2tf[tkn] = 1
    len_heads.append(len(head.split())) #1
    
    body = row[2].lower().strip()
    tkn_para = []
    for para in body.split('<eop>'):
        if para and para != ' ':
            _para = para + '<eop>'
            len_para = len(_para.split())
            len_paras.append(len_para) #2
            tkn_para.append(_para)
    cnt_paras.append(len(tkn_para)) #3
    
    body_split = []
    for tkn in body.split():
        if tkn in tkn2tf: tkn2tf[tkn] += 1
        else: tkn2tf[tkn] = 1
        body_split.append(tkn)
    len_bodys.append(len(body_split)) #4
            
print ((n+1), 'Done')
print (datetime.datetime.now().time())
print ('voca size :', len(tkn2tf))

2021-04-22T02:00:31.043006
17858
02:01:04.792799
voca size : 304846


In [31]:
n

17857

In [32]:
sorted_token = sorted(tkn2tf.items(), key=lambda kv: kv[1], reverse=True)
tkn2idx = {}
for idx, (tkn, _) in tqdm(enumerate(sorted_token)):
    tkn2idx[tkn] = idx + 2
tkn2idx['<unk>'] = 1
tkn2idx[''] = 0
if len(tkn2idx) == len(tkn2tf)+2:
    print (len(tkn2idx), 'No problem')
print 

print ('Show top-10 tkn:')
for tkn, freq in sorted_token[:10]:
    print (tkn,':',freq)
print ('')

304846it [00:00, 408637.98it/s]


304848 No problem
Show top-10 tkn:
<eos> : 1246234
। : 1209771
ও : 112376
করে : 80769
না : 80113
’ : 69678
থেকে : 69205
‘ : 69016
এই : 66790
এ : 60353



In [33]:
#Creating Dic_mincut.txt
with open("Output.txt", "w", encoding="utf-8") as txt_file:
    for tkn, freq in sorted_token:
        txt_file.write(tkn + "\n")

In [34]:
with open(path_processed_data + 'dic_mincutN.txt', 'w', encoding='utf-8') as f:
    for key in tkn2idx.keys():
        f.write(key+'\n')

In [35]:
tkn2tf_mincut5 = {}
for tkn, tf in tkn2tf.items():
    if tf < 30:
        continue
    tkn2tf_mincut5[tkn] = tf
print ('voca size :', len(tkn2tf_mincut5))

voca size : 23212


In [36]:
tkn2tf_mincut5['<EOS>'] = tkn2tf_mincut5['<eos>']
tkn2tf_mincut5['<EOP>'] = tkn2tf_mincut5['<eop>']

del tkn2tf_mincut5['<eos>']
del tkn2tf_mincut5['<eop>']

In [37]:
import operator
sorted_voca = sorted(tkn2tf_mincut5.items(), key=operator.itemgetter(1))

In [38]:
list_voca_mincut = []
list_voca_mincut.append('')   # PAD
list_voca_mincut.append('<UNK>')   # UNK
list_voca_mincut.append('<EOS>')   # EOS
list_voca_mincut.append('<EOP>')   # EOP

for word, idx in sorted_voca:
    if word=='<UNK>' or word=='<EOP>' or word=='<EOS>':
        print("existing word", word)
        continue
    else:
        list_voca_mincut.append(word)
        
len(list_voca_mincut)

existing word <EOP>
existing word <EOS>


23214

In [39]:
with open(path_processed_data + 'dic_mincutN.txt', 'w', encoding='utf-8') as f:
    for i in range(len(list_voca_mincut)):
        f.write(list_voca_mincut[i]+'\n')

In [40]:
dic_voca = {}
for voca in list_voca_mincut:
    dic_voca[voca] = len(dic_voca)

In [41]:
print(dic_voca[''], dic_voca['<UNK>'], dic_voca['<EOS>'], dic_voca['<EOP>'])

0 1 2 3


In [42]:
with open(path_processed_data + 'dic_mincutN.pkl', 'wb') as f:
    cPickle.dump(dic_voca, f)

## read voca from dic

In [43]:
list_voca = []
with open(path_processed_data + 'dic_mincutN.txt', 'r', encoding='utf-8') as f:
    list_voca = f.readlines()
    list_voca = [x.strip() for x in list_voca]

dic_voca = {}
for voca in list_voca:
    dic_voca[voca] = len(dic_voca)

len(dic_voca)

23214

## for data processing

In [44]:
import copy
dic_voca_lower = copy.deepcopy(dic_voca)

In [45]:
dic_voca_lower['<eos>'] = dic_voca_lower['<EOS>']
dic_voca_lower['<eop>'] = dic_voca_lower['<EOP>']

del dic_voca_lower['<EOS>']
del dic_voca_lower['<EOP>']

In [46]:
len(dic_voca_lower)

23214

In [47]:
print(dic_voca_lower[''], dic_voca_lower['<UNK>'], dic_voca_lower['<eos>'], dic_voca_lower['<eop>'])

0 1 2 3


## stats

In [48]:
import csv
import sys
import numpy as np

data= []
with open(path_raw_data + 'train.csv', 'r', encoding='utf-8') as f:
    data_csv = csv.reader(f)
    for row in data_csv:
        data.append(row)

In [49]:
def print_info(data):
    print("mean", np.average(data))
    print("std", np.std(data))
    print("max", np.max(data))
    print("95.xx coverage", np.average(data) +  2*np.std(data) )
    print("99.73 coverage", np.average(data) +  3*np.std(data) )
    print("99.95 coverage", np.average(data) +  3.5*np.std(data) )
    print("99.99 coverage", np.average(data) +  4*np.std(data) )

In [50]:
head = [x[1].strip() for x in data]
head_len = [len(x.split()) for x in head]
print('head_len')
print_info(head_len)

head_len
mean 5.7825624370030235
std 1.9124160333769864
max 16
95.xx coverage 9.607394503756996
99.73 coverage 11.519810537133981
99.95 coverage 12.476018553822476
99.99 coverage 13.43222657051097


In [51]:
body = [x[2].strip() for x in data]

In [52]:
body_len = [len(x.split()) for x in body ]
print('body_len')
print_info(body_len)

body_len
mean 733.8769739052525
std 485.6999437543267
max 4391
95.xx coverage 1705.2768614139059
99.73 coverage 2190.976805168232
99.95 coverage 2433.826777045396
99.99 coverage 2676.6767489225595


In [53]:
context_len = [len(x.split('<EOP>')) for x in body]
print('context_len')
print_info(context_len)

context_len
mean 1.5
std 0.5
max 2
95.xx coverage 2.5
99.73 coverage 3.0
99.95 coverage 3.25
99.99 coverage 3.5


In [54]:
body_sentence = []
for sent in body:
    sent = sent.split('<EOP>')
    body_sentence.extend(sent)
body_len = [ len(x.split()) for x in body_sentence ]    
print('body_sentence')
print_info(body_len)

body_sentence
mean 488.9179826035017
std 343.2417836519407
max 3204
95.xx coverage 1175.401549907383
99.73 coverage 1518.6433335593238
99.95 coverage 1690.2642253852941
99.99 coverage 1861.8851172112645


# encode to numpy

In [55]:
def fit_length(data, max_len_t, max_len_b):
    data_t, data_b = data
    
    list_zeros = np.zeros(max_len_b, 'int32').tolist()
    fl_data_t = []
    for datum in data_t:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_t:
            fl_data_t.append( datum[:max_len_t] )
        else:
            fl_data_t.append( datum + list_zeros[:(max_len_t-_len)] )
            
    fl_data_b = []
    for datum in data_b:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_b:
            fl_data_b.append( datum[:max_len_b] )
        else:
            fl_data_b.append( datum + list_zeros[:(max_len_b-_len)] )
    
    np_data_t = np.asarray(fl_data_t, dtype='int32')
    np_data_b = np.asarray(fl_data_b, dtype='int32')
    
    data = [np_data_t, np_data_b]
    return data

In [56]:
csv_reader = csv.reader(open(path_raw_data + 'train.csv', 'r', encoding='utf-8'))

print (datetime.datetime.now().isoformat())
ids = []
heads = []
bodys = []
labels = []
for n, row in enumerate(csv_reader):
    
#     if n <  3000000:
#         continue

#     if n >=  3000000:
#         continue
        
                
    if (n+1) % 10000 == 0: print (n+1,)
    
    ids.append(row[0])
    labels.append((row[3]))
    
    head = []
    for tkn in row[1].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
            
    heads.append(head)
    
    body = []
    for tkn in row[2].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
            
    bodys.append(body)
    
print (n+1, 'Done')
print (datetime.datetime.now().isoformat()) # ~5 mins

2021-04-22T02:02:50.234979
10000
17858 Done
2021-04-22T02:03:08.710600


In [57]:
print (datetime.datetime.now().isoformat())
[np_heads, np_bodys] = fit_length([heads, bodys], 25, 2100)
print (datetime.datetime.now().isoformat()) # ~3 mins

2021-04-22T02:03:16.559621
2021-04-22T02:03:23.838167


In [58]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/train/train_title.npy'
np.save(t_trainpath, np_heads)
b_trainpath = path_processed_data + '/train/train_body.npy'
np.save(b_trainpath, np_bodys)
l_trainpath = path_processed_data + '/train/train_label.npy'
np.save(l_trainpath, labels)
print (datetime.datetime.now().isoformat())

2021-04-22T02:03:29.534941
2021-04-22T02:03:31.214451


# devset

In [59]:
csv_reader = csv.reader(open(path_raw_data + 'dev.csv', 'r', encoding='utf-8'))

print (datetime.datetime.now().isoformat())
ids_dev = []
heads_dev = []
bodys_dev = []
labels_dev = []
for n, row in enumerate(csv_reader):
    if (n+1) % 10000 == 0: print (n+1,)
    
    ids_dev.append(row[0])
    labels_dev.append((row[3]))
    
    head = []
    for tkn in row[2].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
    heads_dev.append(head)
    
    body = []
    for tkn in row[1].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
    bodys_dev.append(body)
    
print (n+1, 'Done')
print (datetime.datetime.now().isoformat())

2021-04-22T02:03:38.798185
1576 Done
2021-04-22T02:03:40.390927


In [60]:
print (datetime.datetime.now().isoformat())
[np_heads_dev, np_bodys_dev] = fit_length([heads_dev, bodys_dev], 25, 2100)
print (datetime.datetime.now().isoformat()) # ~3 mins

2021-04-22T02:03:45.043491
2021-04-22T02:03:45.612968


In [61]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/dev/dev_title.npy'
np.save(t_trainpath, np_heads_dev)
b_trainpath = path_processed_data + '/dev/dev_body.npy'
np.save(b_trainpath, np_bodys_dev)
l_trainpath = path_processed_data + '/dev/dev_label.npy'
np.save(l_trainpath, labels_dev)
print (datetime.datetime.now().isoformat())

2021-04-22T02:03:49.173451
2021-04-22T02:03:49.203374


# testset

In [62]:
csv_reader = csv.reader(open(path_raw_data + '/test.csv', 'r', encoding='utf-8'))

print (datetime.datetime.now().isoformat())
ids_dev = []
heads_dev = []
bodys_dev = []
labels_dev = []
for n, row in enumerate(csv_reader):
    if (n+1) % 10000 == 0: print (n+1,)
    
    ids_dev.append(row[0])
    labels_dev.append(row[3])
    
    head = []
    for tkn in row[2].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
    heads_dev.append(head)
    
    body = []
    for tkn in row[1].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
    bodys_dev.append(body)
    
print (n+1, 'Done')
print (datetime.datetime.now().isoformat())

2021-04-22T02:03:55.453667
1576 Done
2021-04-22T02:03:57.247874


In [63]:
print (datetime.datetime.now().isoformat())
[np_heads_dev, np_bodys_dev] = fit_length([heads_dev, bodys_dev], 25, 2100)
print (datetime.datetime.now().isoformat()) # ~3 mins

2021-04-22T02:04:00.756492
2021-04-22T02:04:01.284083


In [64]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/test/test_title.npy'
np.save(t_trainpath, np_heads_dev)
b_trainpath = path_processed_data + '/test/test_body.npy'
np.save(b_trainpath, np_bodys_dev)
l_trainpath = path_processed_data + '/test/test_label.npy'
np.save(l_trainpath, labels_dev)
print (datetime.datetime.now().isoformat())

2021-04-22T02:04:04.198294
2021-04-22T02:04:04.223230


# debugset

In [65]:
print (datetime.datetime.now().isoformat())
t_trainpath = path_processed_data + '/debug/debug_title.npy'
np.save(t_trainpath, np_heads_dev[:200])
b_trainpath = path_processed_data + '/debug/debug_body.npy'
np.save(b_trainpath, np_bodys_dev[:200])
l_trainpath = path_processed_data + '/debug/debug_label.npy'
np.save(l_trainpath, labels_dev[:200])
print (datetime.datetime.now().isoformat())

2021-04-22T02:04:07.848539
2021-04-22T02:04:07.938298


In [66]:
with open(path_processed_data + 'dic_mincutN.txt', encoding='utf-8') as f:
    test_list_voca = f.readlines()
    test_list_voca = [x.strip() for x in test_list_voca]

In [67]:
from nlp_vocab import Vocab

In [68]:
tt = Vocab(test_list_voca)

In [69]:
print(tt.index2sent(np_heads_dev[15]))

‘ বাবা আমাদের ছেড়ে চলে গেছেন । <EOS> কানাডার সময় অনুযায়ী ৩১ ডিসেম্বর সন্ধ্যা ছয়টায় তিনি শেষনিশ্বাস ত্যাগ করেন । <EOS> তিনি বিকেলেই <UNK> চলে
